## A/B-тестирование (семинары).
## Урок 8.  
Python применение статистических критериев на практике - пайплайн оценки A/B.

### Задача.  
На сайте запущен А/В тест с целью увеличить доход.  
В приложенном excel файле вы найдете сырые данные по результатам эксперимента – user_id, тип выборки variant_name  
и доход принесенный пользователем revenue.  
  
Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.


#### Решение:  
Найти статистическую значимою разницу среднего дохода от пользователя в $control$ и $variant$.

#### Подготовка данных.

In [172]:
# Импорт бибилиотек.
from typing import Union
from tqdm import tqdm

import pandas as pd
import numpy as np

from scipy import stats
from statsmodels.stats.meta_analysis import effectsize_smd
from statsmodels.stats import proportion
from statsmodels.stats.power import tt_ind_solve_power
from statsmodels.stats.power import zt_ind_solve_power

In [173]:
df = pd.read_excel('C:/Users/BETEPOK/Documents/AB_Testing/HW_8//gb_sem_8_hm.xlsx')

In [174]:
df.head(10)

,USER_ID,VARIANT_NAME,REVENUE
0,737,variant,0.0
1,2423,control,0.0
2,9411,control,0.0
3,7311,control,0.0
4,6174,variant,0.0
5,2380,variant,0.0
6,2849,control,0.0
7,9168,control,0.0
8,6205,variant,0.0
9,7548,control,0.0


In [175]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   USER_ID       10000 non-null  int64  
 1   VARIANT_NAME  10000 non-null  object 
 2   REVENUE       10000 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 234.5+ KB


In [176]:
df.describe()

,USER_ID,REVENUE
count,10000.000000,10000.000000
mean,4981.080200,0.099447
std,2890.590115,2.318529
min,2.000000,0.000000
25%,2468.750000,0.000000
50%,4962.000000,0.000000
75%,7511.500000,0.000000
max,10000.000000,196.010000


Посмотрим на группы отдельно.

In [177]:
control = df[df['VARIANT_NAME'] == 'control'].copy(deep=True)
treatment = df[df['VARIANT_NAME'] == 'variant'].copy(deep=True)

In [178]:
control.describe()

,USER_ID,REVENUE
count,4984.000000,4984.000000
mean,4989.436798,0.129013
std,2905.145109,3.007524
min,2.000000,0.000000
25%,2466.000000,0.000000
50%,4964.500000,0.000000
75%,7576.250000,0.000000
max,10000.000000,196.010000


In [179]:
treatment.describe()

,USER_ID,REVENUE
count,5016.000000,5016.000000
mean,4972.776914,0.070070
std,2876.320625,1.314802
min,3.000000,0.000000
25%,2476.500000,0.000000
50%,4958.500000,0.000000
75%,7415.250000,0.000000
max,10000.000000,58.630000


Наблюдаем, что максимальный доход от пользователя в группе control сильно больше, чем в группе variant.

In [180]:
# Удалим некорректные строки. Посчитаем, сколько типов встречается у каждого пользователя.
v = df.\
    groupby('USER_ID', as_index=False).\
    agg({'VARIANT_NAME': pd.Series.nunique})

In [181]:
v.head(10)

,USER_ID,VARIANT_NAME
0,2,1
1,3,2
2,4,1
3,5,1
4,6,1
5,9,1
6,10,2
7,11,1
8,12,1
9,13,1


In [182]:
# Найдем сколько  уникальных пользователей.
df.USER_ID.nunique()

6324

In [183]:
df = df.groupby(['USER_ID', 'VARIANT_NAME'], as_index=False).agg({'REVENUE': 'sum'})

In [184]:
df.shape

(7865, 3)

In [185]:
# Распределение пользователей по группам.
df.groupby('USER_ID', as_index=False).agg({'VARIANT_NAME': 'count'})['VARIANT_NAME'].value_counts()

1    4783
2    1541
Name: VARIANT_NAME, dtype: int64

Распределение пользователей по группам неравномерное.

In [186]:
# Сформируем новый df.
unique_ids = \
(df
 .groupby('USER_ID', as_index=False)
 .agg({'VARIANT_NAME': 'count'})
 #.['VARIANT_NAME'].value_counts()
 .query('VARIANT_NAME == 1')
 .USER_ID
 .values
 )

In [187]:
df_new = df[df.USER_ID.isin(unique_ids)].copy(deep=True)

In [188]:
df_new.describe()

,USER_ID,REVENUE
count,4783.000000,4783.000000
mean,4994.395777,0.135873
std,2898.618472,3.011392
min,2.000000,0.000000
25%,2476.000000,0.000000
50%,4975.000000,0.000000
75%,7515.000000,0.000000
max,9998.000000,196.010000


In [189]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4783 entries, 0 to 7862
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   USER_ID       4783 non-null   int64  
 1   VARIANT_NAME  4783 non-null   object 
 2   REVENUE       4783 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 149.5+ KB


In [190]:
df_new.head(10)

,USER_ID,VARIANT_NAME,REVENUE
0,2,control,0.0
3,4,variant,0.0
4,5,variant,0.0
5,6,variant,0.0
6,9,variant,0.0
9,11,control,0.0
10,12,control,0.0
11,13,control,0.0
12,15,variant,0.0
15,19,variant,0.0


In [170]:
df_new.VARIANT_NAME.value_counts()

variant    2393
control    2390
Name: VARIANT_NAME, dtype: int64

In [125]:
def continious_result(control: pd.DataFrame,
                      treatment: pd.DataFrame,
                      column: str,
                      n_iters: int = 10_000) -> pd.DataFrame:
    # Статистика по выборкам
    size = control.loc[:, column].shape[0]
    
    control_mean = control.loc[:, column].mean()
    treatment_mean = treatment.loc[:, column].mean()
    
    control_std = control.loc[:, column].std(ddof=1)
    treatment_std = treatment.loc[:, column].std(ddof=1)
    
    # Бутсрап
    booted_diff = []
    for _ in tqdm(range(n_iters)):
        control_sample = control.loc[:, column].sample(n=size, replace=True).values
        treatment_sample = treatment.loc[:, column].sample(n=size, replace=True).values
        booted_diff.append(np.mean(control_sample - treatment_sample))
    
    # Считаем статистику после бустрапа
    md_ci, std_ci = np.mean(booted_diff), np.std(booted_diff, ddof=1)
    left_ci, right_ci = np.percentile(booted_diff, [2.5, 97.5])
    p_value_ci = 2 * (1 - stats.norm.cdf(np.abs(md_ci / std_ci)))
    
    # Считаем мощность эксперимента
    effect_size, _ = effectsize_smd(mean1=treatment_mean, sd1=treatment_std, nobs1=size,
                                    mean2=control_mean, sd2=control_std, nobs2=size)
    power = tt_ind_solve_power(effect_size=effect_size,
                               nobs1=size,
                               alpha=.05,
                               power=None,
                               ratio=1)
    # Формируем отчёт 
    result = pd.DataFrame({'effect_size': effect_size,
                           'alpha': p_value_ci, 
                           'beta': (1-power),
                           'CI': f'[{np.round(left_ci, 3)}, {np.round(right_ci, 3)}]',
                           'difference': md_ci,},
                          index=[column]) 
    return result

In [163]:
fig = px.histogram(df_new,
                   x='REVENUE',
                   color = 'VARIANT_NAME',
                   title='Доход принесенный пользователем.',
                   marginal = 'box',
                   nbins = 100,
                   barmode='overlay')
fig.show()

Экспоненциальное распределение на гистограмме.

In [161]:
control = df_new[df_new['VARIANT_NAME'] == 'control'].copy(deep=True)
treatment = df_new[df_new['VARIANT_NAME'] == 'variant'].copy(deep=True)

In [162]:
continious_result(control, treatment, column='REVENUE')

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:02<00:00, 3665.48it/s]


,effect_size,alpha,beta,CI,difference
REVENUE,-0.040483,0.161851,0.712143,"[-0.004, 0.321]",0.121047


alpha=0.161851 > p_value=0,05, доверительный интервал включает в себя 0  
beta=0.712143 > 0,2=20%  
Моощность теста = 1-0.712143=0,287857  
Распределение пользователей по группам неравномерное.  
1    4783  
2    1541 

#### Ответ:  
Статистически значимая разница среднег дохода от пользователя НЕ НАЙДЕНА.  
Мощность теста мала и могла не выявить статистическу значимую разницу.
Рекомендация:  
- не применять тестируемое изменение и провести тест повторно, перепроверив корректность сплитования трафика,  
- пререрасчитать объем выборки и окно дисперсий,  
- откорректировать шаблон дизайна эксперимента.

